In [1]:
import os
from triadic_closeness.triadic_closeness_classifier import TriadicClosenessClassifier
from metrics.roc import compute_fpr_and_tpr
from sklearn.metrics import roc_curve
from ego_network import EgoNetwork, read_ego_network, get_ego_ids
from triadic_closeness.triadic_closeness import compute_triadic_closeness_for_all_nodes
from triadic_closeness.normalize import normalize_triadic_closeness
from io_helpers.writers import write_ndarray_to_file


DATASET_PATH = '/home/adriana/Documents/master/hands_on_recommender_systems/Project/data/ego-twitter/twitter'
RESULTS_PATH = '/home/adriana/Documents/master/hands_on_recommender_systems/Project/data/ego-twitter/results'
NORMALIZATION_METHODS = ['l1', 'l2', 'max']
MODELS = {
  'tc_n': {'weights_strategy': 'neighbours', 'use_preprocessed_features': False},
  'tc_f': {'weights_strategy': 'features', 'use_preprocessed_features': False},
  'tc_nf': {'weights_strategy': 'neighbours_features', 'use_preprocessed_features': False},
  'tc_fp': {'weights_strategy': 'features', 'use_preprocessed_features': True},
  'tc_nfp': {'weights_strategy': 'neighbours_features', 'use_preprocessed_features': True},
  'tc_jn': {'weights_strategy': 'jaccard_neighbours', 'use_preprocessed_features': False},
  'tc_jf': {'weights_strategy': 'jaccard_features', 'use_preprocessed_features': False},
  'tc_jfp': {'weights_strategy': 'jaccard_features', 'use_preprocessed_features': True},
}

ids = get_ego_ids(DATASET_PATH)
for id in ids:
  print('ID:', id)
  ego_network = read_ego_network(id, DATASET_PATH)
  
  results_directory_for_id = os.path.join(RESULTS_PATH, str(id))
  os.makedirs(results_directory_for_id, exist_ok=True)

  for (key, config) in MODELS.items():
    print(key)
    tc = compute_triadic_closeness_for_all_nodes(ego_network, 
                                                 weights_strategy=config['weights_strategy'],
                                                 use_preprocessed_features=config['use_preprocessed_features'])
    if not len(tc):
      continue
    for norm in NORMALIZATION_METHODS:
      fpr, tpr = compute_fpr_and_tpr(normalize_triadic_closeness(tc, norm), ego_network)
      if fpr is None:
        continue
      write_ndarray_to_file(fpr, os.path.join(results_directory_for_id, f'{key}_{norm}.fpr'))
      write_ndarray_to_file(tpr, os.path.join(results_directory_for_id, f'{key}_{norm}.tpr'))



2022-12-16 14:47:54.291289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 14:47:54.291321: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 14:47:56.571749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-16 14:47:56.571768: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-16 14:47:56.571784: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (adriana-ZenBook-UX391FA-UX391FA): /proc/driver/nvidia/version does not exist
/home/adriana/.local/lib/python3.8/site-packa

ID: 98801140
tc_n
tc_f
tc_nf
tc_fp
tc_nfp
tc_jn
tc_jf
tc_jfp
